<a href="https://colab.research.google.com/github/Miku0204/Michigan-Computer-Vision/blob/main/KNN%20Classifier/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-up code

In [ ]:
user_email = "your github email"
user_name  = "your github user name"

!git config --global user.email {user_email}
!git config --global user.name {user_name}

In [ ]:
repository = "Michigan-Computer-Vision"
git_token = "your github token"

!git clone https://{git_token}@github.com/{user_name}/{repository}

In [ ]:
import os 
repository_path = os.path.join("." , repository)
knn_name        = "KNN Classifier"
knn_path        = os.path.join(repository_path , knn_name)
utils_path      = os.path.join(knn_path        , "utils" )
print(os.listdir(knn_path))
print(os.listdir(utils_path))

In [6]:
import sys
sys.path.append(knn_path)

In [7]:
import utils
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Data visualization

In [ ]:
num_train  = 40000
num_test   = 5000
train_image , train_label , test_image , test_label  = utils.data.data_loading(num_train , num_test)

In [9]:
class_dict = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
help(utils.vis.random_samples)

utils.vis.random_samples(train_image , train_label , class_dict , 10)

# KNN Classifier